# Hashtags

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast

#### Notes

To pull hashtags from captions was simple matter of split each caption by space and selecting strings with '#' at index 0.

To add on hashtags from first comment:

In [82]:
df = pd.read_csv('media_objects1232019.csv')
df = df[1:]
df.drop(['media_object_id', 'media_url'], axis = 1, inplace = True)
#Recast some columns as float to allow for aggregating later
cols_to_cast = ['engagement', 'impressions', 'reach', 'saved']
df[cols_to_cast] = df[cols_to_cast].astype(float)

In [86]:
def get_ht(line):
    return [word[1:] for word in line.split() if word[0] == '#']

#Get hashtags from the caption column
fr_cap = [get_ht(line) for line in df['caption'].astype(str).values]
#Get hashtags from the first comment in the comments column
fr_com = [get_ht(ast.literal_eval(line)['data'][0]['text'])\
                              if type(line) == str else [] for line in df['comments'].values ]
#Extend each list in the 'from captions' list by the values in the 'from comments' list
[fr_cap[i].extend(fr_com_i) for i, fr_com_i in enumerate(fr_com)]

#Add this as a column to the df
df['hashtags'] = fr_cap

In [87]:
#Get an expanded df with individual hashtags
exploded = df.drop(['caption', 'comments', 'media_type'], axis = 1).fillna(0).explode('hashtags')
exploded.reset_index(inplace = True)
#Get unique hashtags
hashtags = 
#Engineer a 'frequency' (aka value count) column value for each hashtag
exploded['frequency'] = [hashtags[tag] if type(tag) == str else 0 for tag in exploded['hashtags'].values]

In [88]:
#94 hashtags end in a period, btw
len([value for value in hashtags.index[:-1] if value[-1] == '.'])

94

In [92]:
num_columns = ['comments_count', 'like_count','engagement','impressions', 'reach', 'saved', 'video_views', 'hashtags']
#Group by hashtags to sum up numerical columns
hashtag_df = exploded[num_columns].groupby(['hashtags']).sum()
#Engineer a column that lists the timestamps for each hashtag
hashtag_df['timestamps'] = exploded.groupby(['hashtags'])['timestamp'].agg(lambda col: ', '.join(col))

In [ ]:
#????
if type(tag) == str else 0 

In [102]:
hashtag_df['frequency'] = [exploded['hashtags'].value_counts()[tag] for tag in hashtag_df.index]

In [103]:
hashtag_df.head()

,comments_count,like_count,engagement,impressions,reach,saved,video_views,timestamps,frequency
hashtags,,,,,,,,,
,0.0,9.0,0.0,0.0,0.0,0.0,0.0,2014-12-25 02:42:40+00:00,1
1.,7.0,81.0,0.0,0.0,0.0,0.0,0.0,"2015-04-29 22:17:15+00:00, 2016-08-15 12:34:41...",2
1875,0.0,3.0,0.0,0.0,0.0,0.0,0.0,2015-02-03 22:51:02+00:00,1
2018,0.0,62.0,63.0,858.0,730.0,1.0,0.0,2017-12-08 19:28:09+00:00,1
3,1.0,20.0,0.0,0.0,0.0,0.0,0.0,2015-04-18 21:11:15+00:00,1


In [99]:
hashtag_df.sort_values(by = ['like_count'], ascending = False)

,comments_count,like_count,engagement,impressions,reach,saved,video_views,timestamps
hashtags,,,,,,,,
cheeseforlife,188.0,19685.0,18115.0,100866.0,82031.0,211.0,1489.0,"2015-04-09 14:21:28+00:00, 2015-06-04 16:29:18..."
linkinbio,453.0,14912.0,13333.0,59536.0,50350.0,377.0,5244.0,"2015-10-14 01:04:25+00:00, 2015-10-17 15:43:02..."
creativityfound,120.0,14629.0,14922.0,94131.0,76559.0,201.0,941.0,"2016-10-21 14:08:46+00:00, 2017-03-22 15:30:15..."
thatcheeseplate,141.0,12005.0,11109.0,76409.0,61726.0,184.0,1079.0,"2015-06-30 13:41:07+00:00, 2015-07-02 16:24:08..."
fravorites,157.0,10165.0,8039.0,69039.0,56157.0,145.0,941.0,"2015-05-05 13:11:29+00:00, 2015-05-13 15:47:54..."
...,...,...,...,...,...,...,...,...
stone,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2015-02-03 20:17:18+00:00
grits,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2015-01-01 19:11:33+00:00
sauerkraut,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2015-01-01 19:11:33+00:00


In [106]:
hashtag_df.to_csv('hashtags.csv')